In [29]:
import sys
sys.path.append('/usr/lib/python3/dist-packages')
import os
import pandas as pd
import datetime
import pandas as pd
import numpy as np

import json
#import shapefile
#from shapely.geometry  import Point, LineString, Polygon, MultiPolygon

from django.conf import settings
#from sqlalchemy import create_engine
os.environ["DJANGO_SETTINGS_MODULE"] = "mysite.settings"
import django
django.setup()
from tqdm import tqdm_notebook, tqdm
import glob

from django.db.models import Q, Sum, Avg

from pyproj import Proj, transform

PROJ3857  = Proj("+init=EPSG:3857")
PROJ4326 = Proj("+init=EPSG:4326")

In [30]:
import requests

In [31]:
from main_map.models import OrganizationNatClass, TimeSquare
import itertools

In [32]:
SERVER_URL = 'http://178.62.250.186'
#SERVER_URL = 'http://127.0.0.1'

In [33]:
SERVER_URL_MSK_DRIVING = SERVER_URL + ':5000/table/v1/driving/'
SERVER_URL_MSK_FOOT =  'http://0.0.0.0:5001/table/v1/foot/'
SERVER_URL_PETER_DRIVING = SERVER_URL + ':5002/table/v1/driving/'
SERVER_URL_PETER_FOOT = SERVER_URL + ':5003/table/v1/foot/'
SERVER_URL_MSK_DRIVING = 'http://0.0.0.0:6000/table/v1/driving/'

In [34]:
# def get_time_3000(lon, lat, coords, root_url):
#     page_url = root_url + '{},{};'.format(lon, lat)
    
#     times = []
#     for coord in coords:
#         page_url = page_url + '{},{};'.format(coord[0], coord[1])
#     page_url = page_url[:-1] + '?sources=0'
#     print(page_url)
#     page = requests.get(page_url)
#     data = page.json()
#     if 'durations' in data:
#         times.extend(data['durations'][0][1:])
#     else:
#         print('no key name duration in data from osrm')
#         times.extend([0]*len(coords))
#     return times
def get_time(lon, lat, coords, mode):

    root_url = {
        'driving': SERVER_URL_MSK_DRIVING,
        'foot': SERVER_URL_MSK_FOOT,
        }[mode]

    times = []
    if len(coords) < 3000:
        times.extend(get_time_3000(lon,lat, coords, root_url))
    else:
        for coord_chunk in tqdm_notebook(np.array_split(coords, len(coords) / 3000)):
            times.extend(get_time_3000(lon, lat, coord_chunk, root_url))

    # Заменить все None на нули
    times = [0 if i is None else i for i in times]
    return times

def get_time_3000(lon, lat, coords, root_url):
    page_url = root_url + '{},{};'.format(lat, lon)
    times = []
    for coord in coords:
        page_url = page_url + '{},{};'.format(coord[0], coord[1])
    page_url = page_url[:-1] + '?sources=0'
    #print(page_url)
    page = requests.get(page_url)
    #print(page)
    data = page.json()
    #print(data)
    if 'durations' in data:
        times.extend(data['durations'][0][1:])
    else:
        print('no key name duration in data from osrm')
        times.extend([0]*len(coords))
    return times


def get_time(lon, lat, coords, mode):
    try:
        # Проверка на Питер
        if lat > 58 and lat > 58:
            root_url = {
                'driving': SERVER_URL_PETER_DRIVING,
                'foot': SERVER_URL_PETER_FOOT,
                }[mode]
        else:
            root_url = {
                'driving': SERVER_URL_MSK_DRIVING,
                'foot': SERVER_URL_MSK_FOOT,
                }[mode]
    except KeyError as e:
        raise ValueError('Undefined mode: {}'.format())
    times = []
    if len(coords) < 3000:
        times.extend(get_time_3000(lon,lat, coords, root_url))
    else:
        for coord_chunk in tqdm_notebook(np.array_split(coords, len(coords) / 3000)):
            times.extend(get_time_3000(lon, lat, coord_chunk, root_url))

    # Заменить все None на нули
    times = [0 if i is None else i for i in times]
    return times

def get_coord_time(x, y, step, rmax, mode):
    coords = get_coords(x, y, step, rmax)
    lat, lon = transform(PROJ3857, PROJ4326, x, y)
    times = get_time(lon, lat, coords, mode)
    return coords, times

def get_coords(x, y, step, rmax):
    coords = itertools.product(np.arange(x - rmax, x + rmax, step),np.arange(y - rmax, y + rmax, step))
    coords = list(coords)
    #print(len(list(coords)))
    #print(list(coords)[0])
    coords = list(map(lambda coord: transform(PROJ3857, PROJ4326, coord[0], coord[1]), coords))
    #print(coords)
    return coords

In [ ]:
df

In [35]:
lat, lon = 55.754638, 37.622024

In [36]:
x, y = transform(PROJ4326, PROJ3857, lon, lat)

In [37]:
step = 300
rmax = 10000
mode = 'foot'

In [38]:
coords, times = get_coord_time(x, y, step, rmax, mode)

In [ ]:
len(coords)

In [22]:
#times = np.array(times) - min(times)

In [23]:
transform(PROJ4326, PROJ3857, coords[0][0], coords[0][1])

(4178064.5542923193, 7499725.274847777)

In [24]:
transform(PROJ4326, PROJ3857, coords[1][0], coords[1][1])

(4178064.5542923193, 7500025.274847784)

In [25]:
7479725.274847778 - 7480025.274847779

-300.00000000186265

In [26]:
import matplotlib.pyplot as plt

In [27]:
TimeSquare.objects.all().delete()

(4489, {'main_map.TimeSquare': 4489})

In [28]:
for i in tqdm_notebook(range(len(times))):
    obj = TimeSquare()
    obj.foot = times[i]
    coord = coords[i]
    obj.x, obj.y = transform(PROJ4326, PROJ3857, coord[0], coord[1])
    obj.save()